# ITS_LIVE STAC catalog search

This is a sample to search velocity granules using the all new ITS_LIVE STAC catalog.

In [2]:
from pystac_client import Client
import time

def search_stac(stac_catalog, page_size=100, filter_list=[], **kwargs):
    """
    Returns only the url to the data asset, for ITS_LIVE a netcdf file.
    """
    catalog = Client.open(stac_catalog)
    search_kwargs = {
        "collections": ["itslive-granules"],
        "limit": page_size,
        **kwargs
    }

    def build_cql2_filter(filters_list):
        if not filters_list:
            return None
        return filters_list[0] if len(filters_list) == 1 else {"op": "and", "args": filters_list}
    if filter_list:
        filters = build_cql2_filter(filter_list)
        search_kwargs["filter"] = build_cql2_filter(filters)
        search_kwargs["filter_lang"] = "cql2-json"

    search = catalog.search(**search_kwargs)
    print(search.get_parameters())
    
    hrefs = []
    pages_count = 0
    proj_matches = 0
    for page in search.pages():
        pages_count += 1
        for item in page:
            if kwargs.get("debug"):
                print(f"fetching page {page_count}")
            for asset in item.assets.values():
                if "data" in asset.roles and asset.href.endswith(".nc"):
                    hrefs.append(asset.href)
        time.sleep(0.1) # we can remove this one, just to avoid overwhelming the server
    print(f"Requested pages: {pages_count}")
    return hrefs

In [3]:
%%time

roi = {
    "type": "Polygon",
    "coordinates": [[
            [
              -35.11660324172296,
              71.70932130492467
            ],
            [
              -32.56940542707364,
              71.70932130492467
            ],
            [
              -32.56940542707364,
              72.26734859624985
            ],
            [
              -35.11660324172296,
              72.26734859624985
            ],
            [
              -35.11660324172296,
              71.70932130492467
            ]
          ]
    ]
}

# Full list of properties can be found by inspecting a STAC item from the collection
# e.g. https://stac.itslive.cloud/collections/itslive-granules/items/S2B_MSIL1C_20250410T133729_N0511_R067_T33XXK_20250410T171508_X_S2A_MSIL1C_20250417T133731_N0511_R067_T33XXK_20250417T205237_G0120V02_P054

filters = [
    {"op": ">=", "args": [{"property": "percent_valid_pixels"}, 50]},
    {"op": "<=", "args": [{"property": "dt_days"}, 8]}, # or date_dt 
    {"op": "=", "args": [{"property": "sat:orbit_state"}, "ascending"]},
]

items = search_stac("https://stac.itslive.cloud",
                    datetime="2010-06-01/2023-06-30",
                    intersects=roi,
                    filter_list=filters,
                    page_size=2000)
len(items)

{'limit': 2000, 'datetime': '2010-06-01T00:00:00Z/2023-06-30T23:59:59Z', 'collections': ('itslive-granules',), 'intersects': {'type': 'Polygon', 'coordinates': [[[-35.11660324172296, 71.70932130492467], [-32.56940542707364, 71.70932130492467], [-32.56940542707364, 72.26734859624985], [-35.11660324172296, 72.26734859624985], [-35.11660324172296, 71.70932130492467]]]}, 'filter': {'op': 'and', 'args': {'op': 'and', 'args': [{'op': '>=', 'args': [{'property': 'percent_valid_pixels'}, 50]}, {'op': '<=', 'args': [{'property': 'dt_days'}, 8]}, {'op': '=', 'args': [{'property': 'sat:orbit_state'}, 'ascending']}]}}, 'filter-lang': 'cql2-json'}
Requested pages: 1
CPU times: user 27.5 ms, sys: 5.67 ms, total: 33.2 ms
Wall time: 1.37 s


110

In [4]:
items[0]

'https://its-live-data.s3.amazonaws.com/velocity_image_pair/sentinel1/v02/N70W030/S1B_IW_SLC__1SDH_20211221T195756_20211221T195823_030128_0398FA_581A_X_S1A_IW_SLC__1SDH_20211227T195838_20211227T195905_041199_04E552_387F_G0120V02_P099.nc'